<a href="https://colab.research.google.com/github/willakuffo/VoiceCon/blob/master/vcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers.core import Dense
import numpy
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [0]:
import sister
embedder = sister.MeanEmbedding(lang = 'en')


Loading model...


In [0]:
with open ('voiceConData.pickle','rb') as D:
  data = pickle.load(D)

In [0]:
data.columns = list(range(data.shape[1]))
data.index = list(range(data.shape[0]))
train_samples = data[list(range(0,data.shape[1]-1))]
train_labels = data[300]



0        door_pv
1      lights_nv
2        door_nv
3      garage_pv
4       water_pv
         ...    
131    garage_nv
132    garage_nv
133      door_pv
134    lights_pv
135      door_nv
Name: 300, Length: 136, dtype: object

In [0]:
#label encoding
encoder = LabelEncoder()
encoded = encoder.fit_transform(train_labels)
LABELS =  np_utils.to_categorical(encoded)


In [0]:
#normalize
sc = MinMaxScaler()
train_data = sc.fit_transform(train_samples)

(136, 300)

In [0]:
def classifier(train_data,labels):
  model = Sequential()
  model.add(Dense(units = train_data.shape[1],input_dim = train_data.shape[1],activation='relu'))
  model.add(Dense(units = 512,activation = 'relu'))
  model.add(Dense(units = 1024,activation = 'relu'))
  model.add(Dense(units = 512,activation = 'relu'))
  model.add(Dense(units = 256,activation = 'relu'))
  model.add(Dense(units = 64,activation = 'relu'))
  model.add(Dense(units = labels.shape[1],activation = 'softmax'))
  return model

model = classifier(train_data,LABELS) 
#model.summary ()  
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
history = model.fit(x = train_data,y = LABELS,validation_split= 0.1,epochs = 20,steps_per_epoch=10,validation_steps=1)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 122 samples, validate on 14 samples
Epoch 1/20





10/10 [==============================] - 2s 174ms/step - loss: 1.4456 - acc: 0.4303 - val_loss: 0.8054 - val_acc: 0.7857
Epoch 2/20
10/10 [==============================] - 0s 42ms/step - loss: 0.3950 - acc: 0.8885 - val_loss: 0.1073 - val_acc: 1.0000
Epoch 3/20
10/10 [==============================] - 0s 43ms/step - loss: 0.0845 - acc: 0.9762 - val_loss: 0.0453 - val_acc: 1.0000
Epoch 4/20
10/10 [==============================] - 0s 43ms/step - loss: 0.0231 - acc: 0.9918 - val_loss: 0.0712 - val_acc: 1.0000
Epoch 5/20
10/10 [==============================] - 0s 41ms/step - loss: 0.0132 - acc: 0.9918 - val_loss: 0.0762 - val_acc: 0.9286
Epoch 6/20
10/10 [==============================] - 0s 42ms/step - loss: 0.0126 - acc: 0.9918 - val_loss: 0.0344 - val_acc: 1.0000
Epoch 7/20
10/10 [==============================] - 0s 42ms/step -

In [0]:
model.save('vcnet.hdf5')

In [0]:
from keras.models import load_model
model = load_model('vcnet.hdf5')

#making labe matches
with open('prediction_matches.pickle','rb') as P:
            matcher = pickle.load(P)


def tell(to_predict):
        embedded = embedder(to_predict)
        embedded = embedded.reshape(-1,1) # reshape for scaling
        sc = MinMaxScaler()
        embedded = sc.fit_transform(embedded)
        embedded = embedded.reshape(1,-1) #reshape for one sample and prediction
        prob,p_class = model.predict(embedded),model.predict_classes(embedded)
        return prob,matcher[p_class[0]],model.predict_classes(embedded)

tell('turn off the water')

(array([[1.3721398e-02, 1.1454125e-02, 2.6081031e-04, 5.7684803e-05,
         4.4938195e-01, 3.7605658e-03, 4.7391167e-01, 4.7451813e-02]],
       dtype=float32), 'water_nv', array([6]))